### DATA MAPPING


In [5]:
import pandas as pd

# Load FILE TWO and process _id column
df2 = pd.read_csv('../proddata/bahay_ugnayan_inventories.csv')
df2['_id_clean'] = df2['_id'].str.replace(r'ObjectId\((.*)\)', r'\1', regex=True)

# Load FILE ONE
df1 = pd.read_excel('../output/mapped_bahay_ugnayan_June26_2025.xlsx', sheet_name='Sheet1')

# Create mapping from FILE TWO
mapping = df2.set_index('accessionId')['_id_clean']

# Map cleaned _id to FILE ONE
df1['_id_clean'] = df1['Accession ID Number'].map(mapping)

# Generate QR code URLs
base_original = "https://teus-goldenberg.rtdigit.net/inventory/view/"
base_new = "https://inven.wpenginepowered.com/view-inventory-item/?accession_id="

df1['qrcode_original'] = df1['_id_clean'].apply(lambda x: base_original + x if pd.notna(x) else "")
df1['qrcode_new'] = base_new + df1['Accession ID Number'].astype(str)

# Remove temporary column and save
df1.drop(columns=['_id_clean'], inplace=True)
df1.to_excel('../output/mapped_with_qr_code/mapped_with_qr_bahay_ugnayan_July02_2025.xlsx', index=False)